# Mandatory Exercise - Session 5

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is to categorize the given pairs, print their most frequent WordNet synset, their corresponding least common subsumer (LCS) and their similarity using the following functions:

- Path Similarity
- Leacock-Chodorow Similarity
- Wu-Palmer Similarity
- Lin Similarity


In [1]:
# initial imports. Could also be done in the PC
import nltk
import numpy as np
import pandas as pd
import itertools

#additional set of imports
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic


#given set of pairs
pairs = [('the', 'DT'), ('man', 'NN'), ('swim', 'VB'), \
         ('with', 'PR'), ('a', 'DT'), ('girl', 'NN'), \
         ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), \
         ('whilst', 'PR'), ('the', 'DT'), ('woman', 'NN'), \
         ('walk', 'VB')]
n = {}
v = {}
aux = {}
freq = []
lcs = []
definition = []

### 1. Most frequent WordNet synsets

Now, for each pair, we will search for their most frequent WordNet sysnset. In the documentation we can find that there are also adjectives and adverbs listed as options but are not used in this given set of pairs. Listing it as a reference: https://wordnet.princeton.edu/documentation/wn1wn

For this given example, we will  select the words where the **POS tag** is **NN (noun)** or **VB (verb)**. We will then store the synsets of those words in two dictionaries *n and v* to preserve their link to the initial value.

The **most frequent word** always **corresponds** to the **01 value of its synset**, so we can shorten the algorithm to speed up the process. 


In [2]:
for e in pairs:
    if e[0] not in n and e[0] not in v:
        if e[1] == 'NN':
            n[e[0]] = wn.synset(e[0]+'.n.01')
        elif e[1] == 'VB':
            v[e[0]] = wn.synset(e[0]+'.v.01')
            
#verification that it is properly stored
for keys,values in n.items():
    print(keys, values)
    
for keys,values in v.items():
    print(keys, values)
            

man Synset('man.n.01')
girl Synset('girl.n.01')
boy Synset('male_child.n.01')
woman Synset('woman.n.01')
swim Synset('swim.v.01')
walk Synset('walk.v.01')


### 2. Least Common Subsumer (LCS)

For each of the pairs that was found to be on WordNet, we now get their corresponding least commmon subsumer. That is the most specific common ancestor (hypernym) of two concepts found in a given ontology. For example, the LCS of moose and kangaroo in WordNet is mammal.

In order to better process the information in one batch, we add the values to a list, indicating if they are a **'noun'** or a **'verb'** and we find their lowest common hypernyms if they belong to the same class -since noun and verbs do not share Hypernyms between eachother-.
<i>http://www.nltk.org/howto/wordnet_lch.html</i>

In [3]:
for key in n:
    freq.append([key,n[key], 'noun'])

for key in v:
    freq.append([key,v[key], 'verb'])

In [4]:
for key in freq:
    row = []
    for alt in freq:
        if key[2] == alt [2]:
            row.append(key[1].lowest_common_hypernyms(alt[1]))
        else:
            row.append(0)
    lcs.append(row)

We then plot the data, which we label with the first value of each element of the dictionary (name of the pair). Within the documentation,we can find that the LCS between the same words is that very same word, we replaced the empty value with a 0 to further increase its visualization and transform all the matrix to a number. *This will prove useful later in the following algorithms*

In [5]:
lcs_np = np.array(lcs)
label = [i[0] for i in freq]
pd.DataFrame(lcs_np, columns = label, index= label)

,man,girl,boy,woman,swim,walk
man,[Synset('man.n.01')],[Synset('adult.n.01')],[Synset('male.n.02')],[Synset('adult.n.01')],0,0
girl,[Synset('adult.n.01')],[Synset('girl.n.01')],[Synset('person.n.01')],[Synset('woman.n.01')],0,0
boy,[Synset('male.n.02')],[Synset('person.n.01')],[Synset('male_child.n.01')],[Synset('person.n.01')],0,0
woman,[Synset('adult.n.01')],[Synset('woman.n.01')],[Synset('person.n.01')],[Synset('woman.n.01')],0,0
swim,0,0,0,0,[Synset('swim.v.01')],[Synset('travel.v.01')]
walk,0,0,0,0,[Synset('travel.v.01')],[Synset('walk.v.01')]


As we can observe from the results, the similarities make semantical sense. 
- Man and girl are connected by adulthood. Same with man and woman
- Girl and boy are connected by person
- Interestingly enough, Woman and boy are connected by person despite of Man and girl being connected by adulthood. 

### 3. Similarity Value

We now proceed with the implementation of the algorithm. We also initiate an empty list to store the results for each algorithm and initialize the values and minimum results. In order to speed the calculations, we initiate the maximum value of each algorithm outside the loop.

The algorithm calculates the results of the 4 algorithms simultaneously, updates the minimum when needed and rounds the value to the nearest three decimals. 

Regarding the Lin Similarity, we also need to compute the Information Content (IC) value, which loads an IC file from the wordnet_ic corpus.

We also initiate the sum values of the matrix, which will be useful to discuss their effectiveness later on.

In [6]:
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

path, lch, wup, lin = [], [], [], []
v1, v2, v3, v4 = 0, 0, 0, 0
min1, min2, min3, min4 = 10, 10, 10, 10

sum1, sum2, sum3, sum4 = 0, 0, 0, 0

#calculating max value of the algorithms outside the loop to preserve them
a = wn.synset('dog.n.01')
max1 = wn.lch_similarity(a, a)
max2 = wn.lch_similarity(a, a)
max3 = wn.wup_similarity(a, a)
max4 = a.lin_similarity(a, semcor_ic)

for key in freq:
    
    #initializing the rows of the matrices
    row1 = []
    row2 = []
    row3 = []
    row4 = []
    
    for alt in freq:
        # adding only if they belong to the same class 'noun' // 'verb'
        if key[2] == alt[2]:
            
            # calculating the result of the algorithm
            v1 = wn.path_similarity(key[1], alt[1])
            v2 = wn.lch_similarity(key[1], alt[1])
            v3 = wn.wup_similarity(key[1], alt[1])
            v4 = key[1].lin_similarity(alt[1], semcor_ic)
            
            # tracking the minimum value in the matrix
            if min1 > v1:
                min1 = v1
            elif min2 > v2:
                min2 = v2
            elif min3 > v3:
                min3 = v3
            elif min4 > v4:
                min4 = v4
                
            #updating the total "value" of the matrix
            sum1 += round(v1,3)
            sum2 += round(v2,3)
            sum3 += round(v3,3)
            sum4 += round(v4,3)
            
            # rounding it and appending it to row
            row1.append(round(v1, 3))
            row2.append(round(v2, 3))
            row3.append(round(v3, 3))
            row4.append(round(v4, 3))
            
        else:
            row1.append(0)
            row2.append(0)
            row3.append(0)
            row4.append(0)
    path.append(row1)
    lch.append(row2)
    wup.append(row3)
    lin.append(row4)
    
#verification that it is properly stored    
print(path)
print(sum1)
print('\n')
print(lch)
print(sum2)
print('\n')
print(wup)
print(sum3)
print('\n')
print(lin)
print(sum4)
print('\n')


[[1.0, 0.25, 0.333, 0.333, 0, 0], [0.25, 1.0, 0.167, 0.5, 0, 0], [0.333, 0.167, 1.0, 0.2, 0, 0], [0.333, 0.5, 0.2, 1.0, 0, 0], [0, 0, 0, 0, 1.0, 0.333], [0, 0, 0, 0, 0.333, 1.0]]
10.232


[[3.638, 2.251, 2.539, 2.539, 0, 0], [2.251, 3.638, 1.846, 2.944, 0, 0], [2.539, 1.846, 3.638, 2.028, 0, 0], [2.539, 2.944, 2.028, 3.638, 0, 0], [0, 0, 0, 0, 3.258, 2.159], [0, 0, 0, 0, 2.159, 3.258]]
53.68


[[1.0, 0.632, 0.667, 0.667, 0, 0], [0.632, 1.0, 0.632, 0.632, 0, 0], [0.667, 0.632, 1.0, 0.667, 0, 0], [0.667, 0.947, 0.667, 1.0, 0, 0], [0, 0, 0, 0, 1.0, 0.333], [0, 0, 0, 0, 0.333, 1.0]]
14.774999999999999


[[1.0, 0.702, 0.798, 0.802, 0, 0], [0.702, 1.0, 0.274, 0.882, 0, 0], [0.798, 0.274, 1.0, 0.308, 0, 0], [0.802, 0.882, 0.308, 1.0, 0, 0], [0, 0, 0, 0, 1.0, 0.466], [0, 0, 0, 0, 0.466, 1.0]]
14.463999999999997




As we can see from our rudimentary prints, the Leacock-Chodorow Similarity could use some normalization so that we can compare its efficiency to the other algorithms. Since we have already calculated the max and the previous loop is finished, the value of min is also final and we can apply the normalization as follows:

<i>x' = (x - min_lch) / (max_lch - min_lch)</i>

In [7]:
#restart the value of the sum since we normalize
sum2 = 0

for row in range(len(lch)):
    for e in range(len(lch[row])):
        if lch[row][e] == 0:
            pass
        else:
            lch[row][e] = round((lch[row][e]-min2)/(max2-min2), 3)
            sum2 += round(lch[row][e],3)
            
print(sum2)

9.356000000000002


<b>What similarity seems better?</b>

First we proceed with each of the individual plots. One think to verify as we run to the matrices is the simetric value of the data. If the values diverge in each side of the diagonal, it already is a big indication of the inestability of the algorithm since the order of the products should not alter the result. 

In [8]:
path_np = np.array(path)
pd.DataFrame(path_np, columns = label, index= label)

,man,girl,boy,woman,swim,walk
man,1.000,0.250,0.333,0.333,0.000,0.000
girl,0.250,1.000,0.167,0.500,0.000,0.000
boy,0.333,0.167,1.000,0.200,0.000,0.000
woman,0.333,0.500,0.200,1.000,0.000,0.000
swim,0.000,0.000,0.000,0.000,1.000,0.333
walk,0.000,0.000,0.000,0.000,0.333,1.000


Another thing to have in mind for the Leacock-Chodorow Similarity is that we have normalized it with the minimum value in our matrix, and not necessarly the min *possible* value.  This means that normalization could change if we were to calculate more broad results. 

In [9]:
lch_np = np.array(lch)
pd.DataFrame(lch_np, columns = label, index= label)

,man,girl,boy,woman,swim,walk
man,1.000,0.226,0.387,0.387,0.000,0.000
girl,0.226,1.000,0.000,0.613,0.000,0.000
boy,0.387,0.000,1.000,0.102,0.000,0.000
woman,0.387,0.613,0.102,1.000,0.000,0.000
swim,0.000,0.000,0.000,0.000,0.788,0.175
walk,0.000,0.000,0.000,0.000,0.175,0.788


In [10]:
wup_np = np.array(wup)
pd.DataFrame(wup_np, columns = label, index= label)

,man,girl,boy,woman,swim,walk
man,1.000,0.632,0.667,0.667,0.000,0.000
girl,0.632,1.000,0.632,0.632,0.000,0.000
boy,0.667,0.632,1.000,0.667,0.000,0.000
woman,0.667,0.947,0.667,1.000,0.000,0.000
swim,0.000,0.000,0.000,0.000,1.000,0.333
walk,0.000,0.000,0.000,0.000,0.333,1.000


In [11]:
lin_np = np.array(lin)
pd.DataFrame(lin_np, columns = label, index= label)

,man,girl,boy,woman,swim,walk
man,1.000,0.702,0.798,0.802,0.000,0.000
girl,0.702,1.000,0.274,0.882,0.000,0.000
boy,0.798,0.274,1.000,0.308,0.000,0.000
woman,0.802,0.882,0.308,1.000,0.000,0.000
swim,0.000,0.000,0.000,0.000,1.000,0.466
walk,0.000,0.000,0.000,0.000,0.466,1.000


To proceed with the discussion of the efectiveness, we now have two tools, the sum of the elements of each of the matrix and the determinants. However, the determinants are not relevant in order to measure the effectiveness in the algorithm. They add and subtract values to calculate the space of the matrix and this is not what we want when evaluating the accuracy of each individual value. 

In [12]:
det_path = round(np.linalg.det(path_np),3)
det_lch = round(np.linalg.det(lch_np),3)
det_wup = round(np.linalg.det(wup_np),3)
det_lin = round(np.linalg.det(lin_np),3)
diff_path = 0.5 - min1
diff_lch = 0.613 - 0.102
diff_wup = 0.947 - min3
diff_lin = 0.882 - min4

# Path Similarity
print('sum of values:', round(sum1,3))
print('determinant:', det_path)  
print('difference in local max-min:', round(diff_path,3), '\n')

# Leacock-Chodorow Similarity
print('sum of values:', round(sum2,3))
print('determinant:', det_lch)  
print('difference in local max-min:', round(diff_lch,3), '\n')

# Wu-Palmer Similarity
print('sum of values:', round(sum3,3))
print('determinant:', det_wup)  
print('difference in local max-min:', round(diff_wup,3), '\n')
      
#Lin Similarity:
print('sum of values:', round(sum4,3))
print('determinant:', det_lin)  
print('difference in local max-min:', round(diff_lin, 3))

sum of values: 10.232
determinant: 0.515
difference in local max-min: 0.333 

sum of values: 9.356
determinant: 0.264
difference in local max-min: 0.511 

sum of values: 14.775
determinant: 0.092
difference in local max-min: 0.614 

sum of values: 14.464
determinant: 0.003
difference in local max-min: 0.574


Besides the sum of values, determinants and difference in local maximum and minimum, we must go back to the basics. In the leacock-similarity matrix, we can see that we have **mistakenly overnormalized the value of the comparison to equal verbs to 0.788**. This is likely due to the difference in the algorithm calculation and it is one of the inconveniences of the normalization. However, due to this very fact, the **normalized would be unsuitable if we were to have a bigger propotion of verbs** and thus, must be discarded for the general approach.

Another thing that we pointed out before is that paths should be the same with identical **switched** starting conditions. As we can see in Wu-Palmer, this is not the case and must also be discarded. 

In [13]:
print(wn.synset('woman.n.01').wup_similarity(wn.synset('girl.n.01')))
print(wn.synset('girl.n.01').wup_similarity(wn.synset('woman.n.01')))

0.9473684210526315
0.631578947368421


Because of this, our decision is now between **Path Similarity** and the **Lin Similarity**. As we can see from the combined analysis of the sum of values and the difference between local maximums and minimums, **Lin gives more accurate scores**, yielding **higher numbers in the overall matrix (total sum: 14.464)** and having a **greater difference between the similar and non-similar words (0.574)**. 

## Conclusions

Over the development of this lab, we have calculated the most frequent WordeNet synsets and their least common subsumers. We have also analyzed their similarity value according to 4 different algorithms and have appllied normalization when needed. 

Another thing to highlight is that the HiddenMarkovModel looks really efficient timewise. It is likely that its just not in the desired set of conditions regarding the amount of data to perform correctly. 

Based on our analysis, we have selected the **Lin-Similarity as the best performing algorithm**. 
